In [2]:
import ee
import geemap
import os

In [3]:
map =geemap.Map()
map

Enter verification code:  4/1AZEOvhU1v43O62tNKfNazfP58KWKaJVlPl96iClbCE3KbPFSOmGm_K83wRU



Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select([0,1,2,3,4,6])
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

hyperion = ee.ImageCollection('EO1/HYPERION').filter(
    ee.Filter.date('2015-01-01', '2015-03-01')
)
hyperion_vis = {
    'min': 1000.0,
    'max': 14000.0,
    'gamma': 2.5,
}
map.addLayer(hyperion, hyperion_vis, 'EO1/HYPERION');

In [ ]:
map.set_plot_options(add_marker_cluster=True, overlay=None)

In [ ]:
m.plot_demo()

In [ ]:
## Adding shapefile
tn_shp = 'data/trentino_shp.shp'
shapefile = geemap.shp_to_ee(tn_shp)
map.addLayer(shapefile, {}, 'TN_Shapefile')

In [ ]:
## export shapefile as kml
geemap.ee_export_vector(shapefile, filename= 'data/trentino_shp.kml')

In [ ]:
## split map window
map =geemap.Map()
map.split_map(left_layer='ESA WorldCover 2020', right_layer='ESA WorldCover 2021')
map

In [ ]:
basemaps = geemap.basemaps.keys()
print(basemaps)

In [ ]:
## Earth engine dataset

img_collection = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
print(img_collection.aggregate_array('system:id').getInfo())

In [ ]:
aafc_2009 = ee.Image('AAFC/ACI/2009').select('landcover')
aafc_2020 = ee.Image('AAFC/ACI/2020').select('landcover')

left_layer = geemap.ee_tile_layer(aafc_2009, {}, 'AAFC 2009')
right_layer = geemap.ee_tile_layer(aafc_2020, {}, 'AAFC 2020')

map = geemap.Map()
map.split_map(left_layer, right_layer)
map

In [ ]:
## clip ee image to shapefile
map = geemap.Map()
map

In [ ]:

image = ee.Image('USGS/SRTMGL1_003')
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
map.addLayer(image, vis_params, 'SRTM DEM')

states = 'data/trentino_shp.shp'
states_shp = geemap.shp_to_ee(states)
map.addLayer(states_shp, {}, 'Trentino')


In [ ]:
## clip with shapefile

roi = image.clip(states_shp)
map.addLayer(roi, vis_params, 'Clipped image')

"""
## clip with drawing feature

map.draw_features

"""

In [ ]:
map.draw_last_feature
roi = ee.FeatureCollection(map.draw_features)
map.addLayer(roi, {}, "Selected states")

In [ ]:
clipped_image = image.clip(roi)
map.addLayer(clipped_image, vis_params, 'Clipped image')

In [ ]:
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 0.7)

In [45]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                [-115.413031, 35.889467],
                [-115.413031, 36.543157],
                [-114.034328, 36.543157],
                [-114.034328, 35.889467],
                [-115.413031, 35.889467],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

In [46]:
## Download image collection as GeoTIFF
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename= os.path.join(out_dir, 'landsat_tn.tif')
image = image.clip(roi).unmask()
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

Generating URL ...
An error occurred while downloading.
Element.geometry: Unable to use a collection in an algorithm that requires a feature or image. This may happen when trying to use a collection of collections where a collection of features is expected; use flatten, or map a function to convert inner collections to features. Use clipToCollection (instead of clip) to clip an image to a collection.


In [94]:
### Zonal statistcs

map = geemap.Map()
map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [79]:
dem = ee.Image('USGS/SRTMGL1_003')
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
map.addLayer(image, vis_params, 'SRTM DEM')

landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select([0,1,2,3,4,6])
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

states = 'data/tn1.shp'
states_shp = geemap.shp_to_ee(states)
states_shp = ee.FeatureCollection(states_shp)
map.addLayer(states_shp, {}, 'Trentino')

In [80]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_stats = os.path.join(out_dir, 'dem_stats.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, states_shp, out_dem_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\hfz_a\Downloads\dem_stats.csv


In [102]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
landcover = ee.Image(dataset.select('forest_type'))
map.addLayer(landcover, {}, 'LULC')

states = 'data/tn1.shp'
states_shp = geemap.shp_to_ee(states)
states_shp = ee.FeatureCollection(states_shp)
map.addLayer(states_shp, {}, 'Trentino')


In [103]:
map.add_legend(builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')